Getting Pulses Using `pyRawIO` 
==============================

The SuperCDMS library `pyRawIO` allows us to read raw MIDAS DAQ files directly to get the raw pulses that our analysis RQs are based on. The library is hosted on the GitLab page for SuperCDMS [[GitLab:pyRawIO][GitLab:pyRawIO]]. The library `pyRawIO` depends on other libraries like `CDMSbats` and `ROOT`-- those are sometimes difficult to install so these instructions focus on using a `singularity` container [[Singularity][Singularity]] where all of that stuff is already pre-installed and all you need to run this is to install `singularity` and have the image files (which are maintained by SuperCDMS [[Singularity-Images][Singularity-Images]]). 

[GitLab:pyRawIO]: https://gitlab.com/supercdms/DataHandling/pyRawIO "GitLab:pyRawIO"
[Singularity]: https://docs.sylabs.io/guides/latest/user-guide/introduction.html "Singularity" 
[Singularity-Images]: https://confluence.slac.stanford.edu/display/CDMS/Using+Containerized+CDMS+Offline+Releases#UsingContainerizedCDMSOfflineReleases-WheretogettheSingularityimagesfrom? "SCDMS Singularity Image Registry"

Singularity
-----------

To begin using singularity you will probably have to install it, but test if you have it by typing `singularity --help` in the terminal; if that returns the help menu then you already have it! Singularity depends on go [[go][go]], so if you don't have it you will have to install that first. Install `singularity` and `go` according to the instructions in this repository [[example_scripts/singularity][sing-install]].

Here are some useful singularity images on disk on the UMN cluster:

 - `/data/chocula/villaa/cdmsfull_latest.sif` (don't remember exactly the SuperCDMS release tag, V04-12?)
 - `/data/chocula/villaa/cdmsfull_V04-07-00.sif`  
 - `/data/chocula/villaa/cdmsfull_V04-13-00.sif`
 
Before using `singularity` you should have the correct environmental variables set, this could be:

```
export GOCACHE="/data/chocula/villaa/.cache/go"
export GOMODCACHE="/data/chocula/villaa/.cache/go/pkg/mod"
export GOPATH="/data/chocula/villaa/go"
export CCACHE_DIR="/data/chocula/villaa/.ccache"
export PATH=$PATH:/data/chocula/villaa/goinstall/go/bin
export SINGULARITY_BINDPATH=/data
```

Most of these set up the paths and cache directories for the language `go` itself. Make the cache directories something that has some space. On the UMN cluster my home directory is severely limited so I used the data directory `/data/chocula/villaa`, other users should be able to make their own directory in `/data/chocula` if they don't already have one. Contact Anthony Villano to get help with setting that up if you don't have permission. 

Open a shell in `singularity` then you will have access to the code environments above created for SuperCDMS and having `pyRawIO`, `CDMSbats` and `ROOT` already installed.

```
$ singularity shell /data/chocula/villaa/cdmsfull_V04-13-00.sif 
WARNING: DEPRECATED USAGE: Environment variable SINGULARITY_BINDPATH will not be supported in the future, use APPTAINER_BINDPATH instead
Singularity> 
```

We also have the ability to run a Jupyter Labs server inside the `singularity` container and expose a web port that can be accessed by a browser outside. Many users typically work from remote locations, so below is the procedure for getting a singularity container running with a Jupyter Labs server and accessing it from a web browser. 

 1. Connect to UMN via VPN
 2. ssh to a UMN machine (maybe `cdms2.spa.umn.edu`) and map port 8889 to your localhost
 3. Spin up a `singularity` instance with a Jupyter Labs server running on port 8889 (this port should not conflict with any running Jupyter Lab instances on the UMN machine or your local machine)
 4. Open a web browser on your local machine with the address `https://localhost:8889`
 
 The first step is accomplished through your VPN client and two-factor authentication as normal. The second step can be accomplished (once the VPN connection is established) via this command on your localhost:
 
 ```
 $ ssh -L 8889:localhost:8889 user@cdms2.spa.umn.edu
 ```

The third step is accomplished in the newly opened shell to the UMN computer `cdms2.spa.umn.edu`. Use the following command:

```
$ singularity run /data/chocula/villaa/cdmsfull_latest.sif --no-browser --port=8889
```

After this step the terminal will display status updates for the `singularity` container that is running. You can check there to see if there are problems or if it's stalled (hit enter and you should see a new line if it's still running). The main thing that can stall the `singularity` instance is the VPN or other connections dropping. If it is the first time you've run this command you will see some output after it that gives you an authentication key to get into the Jupyter Lab instance for the first time (step 4). 

[go]: https://go.dev/doc/install "GO programming language"
[sing-install]: https://github.com/villano-lab/R76Analysis/tree/master/example_scripts/singularity "Installing Singularity"

Loading In Events
-----------------

The toolchain that was developed involves getting a text-file list of series/event numbers from our processed data. That list will be passed to the script `singularity_pulseSave.py` and it will produce a file called `pulses.pkl` that contains a data structure with all the requested pulses in it. 

`singularity_pulseSave.py`
-------------------------

Reading Back Events
-------------------